In [1]:
%reload_ext autoreload
%autoreload 2


import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
from pathlib import Path

import sys
sys.path.append("..")
import main
import utils


In [2]:
models = sorted(list(utils.dirtodict('../../data/CMIP6/').keys()))[1:]

In [ ]:
def open_and_process_dataset(files, time_slice=None):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        ds = xr.open_mfdataset(files, combine="by_coords")
        if time_slice:
            ds = ds.sel(time=ds.time.dt.year.isin(list(range(*time_slice))))
    return ds

baseline_year_length = 20
baseline_type = "moving_baseline"

models = ['EC-Earth3-Veg',]
for model in models:
    print('\n')
    print(model)
    data_path = Path(f"../../data/CMIP6/{model}/")
    dir_dict = utils.dirtodict(data_path)
    variants = sorted(list(dir_dict.keys())[:-1])

    for variant in variants:
        print(variant)
        files_hist = dir_dict[variant]["tos"]["historical"][".files"]
        ds_hist = open_and_process_dataset(files_hist, time_slice=(1982, 2015))

        for experiment in ["ssp126", "ssp585"]:
            print(experiment)

            output_dir = Path(f"../../results/MHW/CMIP6/{model}/{variant}/{baseline_type}/")
            output_dir.mkdir(parents=True, exist_ok=True)
            output_path = output_dir / f"MHW_metrics_{experiment}.nc"

            if os.path.exists(output_path):
                print(f"Skipping")
                continue

            files_ssp = dir_dict[variant]["tos"][experiment][".files"]
            ds_ssp = open_and_process_dataset(files_ssp, time_slice=(2015, 2100))

            ds = xr.concat([ds_hist, ds_ssp], dim="time")

            #Cropping for testing
            # ds = ds.where((ds.longitude <= 10) & (ds.longitude >=5))
            # ds = ds.sel(time=ds.time.dt.year.isin(list(range(1982, 1993))))

            ds_mhw = main.MHW_metrics_cmip(ds, baseline_year_length, baseline_type=baseline_type)

            ds_mhw.to_netcdf(output_path)

In [31]:
y=2015
d = ds.sel(time = ds.time.dt.year.isin(list(range(y, y+1))))
for a in d.time:
    print(a.dt.dayofyear)

<xarray.DataArray 'dayofyear' ()>
array(2)
Coordinates:
    time     object 2015-01-02 00:00:00
<xarray.DataArray 'dayofyear' ()>
array(3)
Coordinates:
    time     object 2015-01-03 00:00:00
<xarray.DataArray 'dayofyear' ()>
array(4)
Coordinates:
    time     object 2015-01-04 00:00:00
<xarray.DataArray 'dayofyear' ()>
array(5)
Coordinates:
    time     object 2015-01-05 00:00:00
<xarray.DataArray 'dayofyear' ()>
array(6)
Coordinates:
    time     object 2015-01-06 00:00:00
<xarray.DataArray 'dayofyear' ()>
array(7)
Coordinates:
    time     object 2015-01-07 00:00:00
<xarray.DataArray 'dayofyear' ()>
array(8)
Coordinates:
    time     object 2015-01-08 00:00:00
<xarray.DataArray 'dayofyear' ()>
array(9)
Coordinates:
    time     object 2015-01-09 00:00:00
<xarray.DataArray 'dayofyear' ()>
array(10)
Coordinates:
    time     object 2015-01-10 00:00:00
<xarray.DataArray 'dayofyear' ()>
array(11)
Coordinates:
    time     object 2015-01-11 00:00:00
<xarray.DataArray 'dayofyear' ()>
arra

In [23]:
7299/365

19.997260273972604

In [34]:
model = "MPI-ESM1-2-LR"
data_path = Path(f"../data/CMIP6/{model}/")
dir_dict = utils.dirtodict(data_path)
variants = sorted(list(dir_dict.keys())[:-1])
files_hist = dir_dict[variants[0]]["tos"]["historical"][".files"]
ds_hist = open_and_process_dataset(files_hist, time_slice=(1982, 2015))

In [9]:
np.shape(ds_hist.latitude.values)

(22, 38)

In [11]:
model = "CESM2"
data_path = Path(f"../data/CMIP6/{model}/")
dir_dict = utils.dirtodict(data_path)
variants = sorted(list(dir_dict.keys())[:-1])
files_hist = dir_dict[variants[0]]["tos"]["historical"][".files"]
ds_hist = open_and_process_dataset(files_hist, time_slice=(1982, 2015))

In [27]:
ds_hist.coords
#np.shape(ds_hist.latitude.values)

Coordinates:
    latitude   (nlat, nlon) float64 dask.array<chunksize=(38, 41), meta=np.ndarray>
    longitude  (nlat, nlon) float64 dask.array<chunksize=(38, 41), meta=np.ndarray>
  * nlat       (nlat) int32 277 278 279 280 281 282 ... 309 310 311 312 313 314
  * nlon       (nlon) int32 31 32 33 34 35 36 37 38 ... 64 65 66 67 68 69 70 71
  * time       (time) object 1982-01-01 00:00:00 ... 2014-12-31 00:00:00

In [25]:
lat_simple = ds_hist.latitude.isel(nlon=0).values

In [32]:
latlon_inx = ds_hist.latitude.dims
'nlat' in latlon_inx or 'nlon' in latlon_inx

True

In [33]:
latlon_inx

('nlat', 'nlon')